# Web Crawling Project 2
## '뉴스 기사 크롤링하기'

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# naver 뉴스

In [250]:
driver = webdriver.Chrome('chromedriver.exe')

driver.get('http://naver.com')

driver.find_element_by_id('query').send_keys('동성제약')
driver.find_element_by_id('search_btn').click()

driver.find_element_by_css_selector('div.lnb_menu').find_element_by_link_text(
    '뉴스').click()

driver.find_element_by_id('_search_option_btn').click()
menu = driver.find_elements_by_css_selector('ul.option_menu > li.menu')
menu[2].click()
menu[2].find_element_by_link_text('제목').click()

current_window = driver.current_window_handle

data = {'title': [], 'date': [], 'company': [], 'context': [], 'reporter': []}
for i in range(50):
    for naver_news in driver.find_elements_by_link_text('네이버뉴스'):
        naver_news.click()
        driver.switch_to.window(driver.window_handles[1])
        html = driver.page_source
        soup = bs(html, 'html.parser')
        data['title'].append(soup.select('h3.tts_head')[0].text.strip())
        data['date'].append(soup.select('span.t11')[0].text.strip())
        soup.find('div', id='articleBodyContents').script.extract()
        data['context'].append(
            soup.find('div', id='articleBodyContents').text.strip())
        driver.close()
        driver.switch_to.window(current_window)
    driver.find_element_by_link_text('다음페이지').click()

In [251]:
for i in range(50):
    for naver_news in driver.find_elements_by_link_text('네이버뉴스'):
        naver_news.click()
        driver.switch_to.window(driver.window_handles[1])
        html = driver.page_source
        soup = bs(html, 'html.parser')
        data['title'].append(soup.select('h3.tts_head')[0].text.strip())
        data['date'].append(soup.select('span.t11')[0].text.strip())
        soup.find('div', id='articleBodyContents').script.extract()
        data['context'].append(
            soup.find('div', id='articleBodyContents').text.strip())
        driver.close()
        driver.switch_to.window(current_window)
    driver.find_element_by_link_text('다음페이지').click()

In [348]:
driver.switch_to.window(current_window)

In [352]:
%%time

for i in range(120):    
    for naver_news in driver.find_elements_by_link_text('네이버뉴스'):
        naver_news.click()
        driver.switch_to.window(driver.window_handles[1])
        html = driver.page_source
        soup = bs(html, 'html.parser')
        if len(soup.select('h3.tts_head')) == 0:
            driver.close()
            driver.switch_to.window(current_window)
            break
        data['title'].append(soup.select('h3.tts_head')[0].text.strip())
        data['date'].append(soup.select('span.t11')[0].text.strip())
        soup.find('div', id = 'articleBodyContents').script.extract()
        data['context'].append(soup.find('div', id = 'articleBodyContents').text.strip())
        driver.close()
        driver.switch_to.window(current_window)
    driver.find_element_by_link_text('다음페이지').click()

TypeError: object of type 'NoneType' has no len()

In [355]:
df = pd.DataFrame(data, columns = ['title', 'date', 'context'])
df

,title,date,context
0,"[하반기 특징주]⑤'숨고르기' 동성제약, 다시 랠리 이어가나",2018-09-22 06:11,하반기 들어 153% ↑…지난 5일 연중 최고치 기록연구개발비 자산으로 인식한 새 ...
1,"동성제약, ‘2018 장애인 아시안게임’ 국가대표 후원",2018-09-21 10:15,/사진=동성제약동성제약이 대한장애인체육회와 힘을 합쳐 장애인 아시안게임 국가대표 선...
2,"[한경로보뉴스] '동성제약' 5% 이상 상승, 거래량 큰 변동 없음. 전일 24% 수준",2018-09-20 10:14,"20일 오전 10시 11분 현재 동성제약은 전 거래일보다 5.23% 오른 38,20..."
3,"[한경로보뉴스] '동성제약' 5% 이상 상승, 2018.2Q, 매출액 224억(+4...",2018-09-19 09:29,동성제약이 장 초반 부터 전일 대비 +5%이상 상승하고 있다. 19일 오전 09시 ...
4,"동성제약, 전일 대비 약 7% 상승한 45,500원",2018-09-12 11:00,[한국경제TV 라이온봇 기자][그림 1] 최근 3개월 간 주가 추세 및 기간별 수익...
5,"[특징주] 동성제약, 사흘째 상승세… LED 암치료 기대감",2018-09-12 10:28,동성제약이 3거래일 연속 상승세다.12일 오전 10시15분 현재 동성제약은 전 거래...
6,[fnRASSI]동성제약 전일대비 10.32% 상승,2018-09-12 10:14,"동성제약(002210)은 현재 주가가 전일 대비 10.32% 상승한 47,050원 ..."
7,"[한경로보뉴스] '동성제약' 5% 이상 상승, 2018.2Q, 매출액 224억(+4...",2018-09-12 09:50,동성제약이 장 초반 부터 전일 대비 +5%이상 상승하고 있다. 12일 오전 09시 ...
8,"<유>동성제약, 전일 대비 7.03% 상승.. 일일회전율은 2.77% 기록",2018-09-12 09:47,[서울경제] 동성제약(002210)은 12일 오전 9시 47분 현재 7.03% 오른...
9,"[한경로보뉴스] '동성제약' 5% 이상 상승, 2018.2Q, 매출액 224억(+4...",2018-09-11 11:26,"11일 오전 11시 24분 현재 동성제약은 전 거래일보다 5.85% 오른 43,40..."


In [368]:
df.to_csv('동양제약_naver_news.csv')

In [375]:
df.drop_duplicates('context', keep = 'first', inplace = True)
df.reset_index(inplace = True, drop = True)
df.to_csv('동양제약_naver_news.csv')

In [376]:
df

,index,title,date,context
0,0,"[하반기 특징주]⑤'숨고르기' 동성제약, 다시 랠리 이어가나",2018-09-22 06:11,하반기 들어 153% ↑…지난 5일 연중 최고치 기록연구개발비 자산으로 인식한 새 ...
1,1,"동성제약, ‘2018 장애인 아시안게임’ 국가대표 후원",2018-09-21 10:15,/사진=동성제약동성제약이 대한장애인체육회와 힘을 합쳐 장애인 아시안게임 국가대표 선...
2,2,"[한경로보뉴스] '동성제약' 5% 이상 상승, 거래량 큰 변동 없음. 전일 24% 수준",2018-09-20 10:14,"20일 오전 10시 11분 현재 동성제약은 전 거래일보다 5.23% 오른 38,20..."
3,3,"[한경로보뉴스] '동성제약' 5% 이상 상승, 2018.2Q, 매출액 224억(+4...",2018-09-19 09:29,동성제약이 장 초반 부터 전일 대비 +5%이상 상승하고 있다. 19일 오전 09시 ...
4,4,"동성제약, 전일 대비 약 7% 상승한 45,500원",2018-09-12 11:00,[한국경제TV 라이온봇 기자][그림 1] 최근 3개월 간 주가 추세 및 기간별 수익...
5,5,"[특징주] 동성제약, 사흘째 상승세… LED 암치료 기대감",2018-09-12 10:28,동성제약이 3거래일 연속 상승세다.12일 오전 10시15분 현재 동성제약은 전 거래...
6,6,[fnRASSI]동성제약 전일대비 10.32% 상승,2018-09-12 10:14,"동성제약(002210)은 현재 주가가 전일 대비 10.32% 상승한 47,050원 ..."
7,7,"[한경로보뉴스] '동성제약' 5% 이상 상승, 2018.2Q, 매출액 224억(+4...",2018-09-12 09:50,동성제약이 장 초반 부터 전일 대비 +5%이상 상승하고 있다. 12일 오전 09시 ...
8,8,"<유>동성제약, 전일 대비 7.03% 상승.. 일일회전율은 2.77% 기록",2018-09-12 09:47,[서울경제] 동성제약(002210)은 12일 오전 9시 47분 현재 7.03% 오른...
9,9,"[한경로보뉴스] '동성제약' 5% 이상 상승, 2018.2Q, 매출액 224억(+4...",2018-09-11 11:26,"11일 오전 11시 24분 현재 동성제약은 전 거래일보다 5.85% 오른 43,40..."


https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=022&aid=0002671699

이 놈에서 자꾸 오류가 발생하는데.... 뭔지를 모르겟다

# 발생한 오류
### 1. 인터넷 뉴스 회사마다 html형식이 달라 크롤링에 제약이 생겼다.
 - 해결 못 함

### 2. pd.read_csv() 과정에서 'Initializing from file failed' 오류가 발생
 - pandas의 자체 버그이며 해결방법으로는 engine = 'python' 옵션을 추가해주면 된다.
 
### 3. for구문 중간에 자꾸 list Index out of range 오류가 발생
 - 이거는 단지 select된 element내에 아무것도 없었기 때문에 [0]에서 오류가 발생
 - if구문을 통해 조건을 걸어 무시할 수 있도록 조치함

# 새로 배운 것
### 1. Multiple class인 element 찾기
 - css_selector(.class.class)

### 2. BeautifulSoup를 통해 parsing된 문서에서 id로 찾기(select 말고)
 - soup.find('tag', id = '')
 - 참고https://stackoverflow.com/questions/25614702/get-contents-of-div-by-id-with-beautifulsoup
 
### 3. 글 중간에 있는 특정 tag를 제외한 상태로 긁어오기
 - soup.find().tag.extract() : tag를 extract()한 상태가 된다.(soup 안에서 해당 tag가 삭제됨)
 - 참고https://stackoverflow.com/questions/48877343/exclude-tags-with-beautifulsoup
 - 제외한다는 말을 'exclude'라고 표현함
 
### 4. DataFrame 내에 중복되는 row 삭제하기
 - pd.drop_duplicates(column_label, keep = 'first')
 - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop_duplicates.html